In [157]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [158]:
# !pip install pydantic --upgrade


In [159]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI

from langchain.tools import tool

In [160]:

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI


In [161]:
class Extract_toolArgs(BaseModel):
    a: str = Field(description="Company name")
    b: str = Field(description="item")
    c: str = Field(description="Year_1")
    d: str = Field(description="Year_2")

In [162]:
class Extracted_toolArgs(BaseModel):
    year : list = Field(description="List of years")
    value : list = Field(discription = "List of Values")

In [163]:
import json
@tool(args_schema=Extract_toolArgs)
def Extract_Tool(a: str, b: str, c:str, d:str) -> Extract_toolArgs:
    """extracts specific data for a company from Year_1 to Year_2."""
    with open("data_soo.json","r") as json_file:
        data = json.load(json_file)
    
    a = a.upper()
    b = b.lower()
    json_data = next(item for item in data[a] if item["Item"] == b)
    
    year_data =[]
    cash_data = []
    
    # Ensure c and d represent years in the correct order
    if int(c) > int(d):
        c, d = d, c
    
    for year in range(int(c), int(d) + 1):
        value = json_data.get(str(year))
        if value is not None:
            cash_data.append(value)
            year_data.append(year)

            
    
    if len(cash_data) == 0:
        raise ValueError(f"Data for the years {c} to {d} is not available.")
    
    return cash_data,year_data




In [164]:
class GrowthToolArgs(BaseModel):
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of sales data")


In [165]:
@tool(args_schema=GrowthToolArgs)
def Growth_Tool(sales_data: list, year_data: list) -> list[float]:
    """
    Calculates year-on-year (YoY) growth for 'Net Sales' for a specific company.
    This function receives year data and sales data list.

    Arguments:
    - sales_data: List of dictionaries containing year and corresponding sales.
    - year_data: List of years corresponding to the sales data.
    
    Returns:
    - A list of floats representing the YoY growth percentage for each year compared to the previous year.
    """
    # Ensure there is sufficient data for year-on-year calculation
    if len(sales_data) < 2:
        return ["Not enough data to calculate year-on-year sales growth."]
    
    yoy_growth = []
    
    # Sort the sales data by year to ensure proper calculation
    sales_data_sorted = sorted(zip(year_data, sales_data), key=lambda x: x[0])

    # Extract sorted years and sales
    sorted_years, sorted_sales = zip(*sales_data_sorted)

    for i in range(1, len(sorted_years)):
        current_year = sorted_years[i]
        previous_year = sorted_years[i - 1]

        current_year_sales = sorted_sales[i]
        previous_year_sales = sorted_sales[i - 1]

        if previous_year_sales is not None and current_year_sales is not None:
            # Calculate YoY growth
            growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
            yoy_growth.append(growth)
        else:
            yoy_growth.append(None)  # If data is missing
    
    return yoy_growth


In [166]:
class GrossMarginToolArgs(BaseModel):
    gross_margin_data : list = Field(description="list of Gross margin data")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of Gross margin data")

In [167]:
@tool(args_schema=GrossMarginToolArgs)
def GrossmmarginpercTool(gross_margin_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates percentage of Gross Margin.
    This function receives year data and sales data list and Gross margin.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    -gross_margin_data: List of corresponding gross margin data.
    
    Returns:
    - A list of floats representing the YoY gross margin percentage for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(gross_margin_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, gross_margin_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_gross_margins, sorted_sales = zip(*data_sorted)

    yoy_gross_margin_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            gross_margin_percentage = (sorted_gross_margins[i] / sorted_sales[i]) * 100
            yoy_gross_margin_percentage.append(gross_margin_percentage)
        else:
            yoy_gross_margin_percentage.append(None)  # If data is missing or invalid

    return yoy_gross_margin_percentage

In [168]:
class expensespercArgs(BaseModel):
    t_o_e_data : list = Field(description="list of total operating expenses data")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of total operating expenses data")

In [169]:
@tool(args_schema=expensespercArgs)
def ExpensespercTool(t_o_e_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates expenses percentage sales.
    This function receives year data and sales data list and total operating expenses.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - t_o_e_data: List of corresponding total operating expenses data.
    
    Returns:
    - A list of floats representing the  expenses percentage sales for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(t_o_e_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, t_o_e_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_t_o_e_data, sorted_sales = zip(*data_sorted)

    yoy_expenses_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            expenses_percentage = (sorted_t_o_e_data[i] / sorted_sales[i]) * 100
            yoy_expenses_percentage.append(expenses_percentage)
        else:
            yoy_expenses_percentage.append(None)  # If data is missing or invalid

    return yoy_expenses_percentage

In [170]:
class TaxratesArgs(BaseModel):
    Provision_of_IT_data : list = Field(description="list of provision for income taxes data")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of total operating expenses data")

In [171]:
@tool(args_schema=TaxratesArgs)
def TaxrateTool(Provision_of_IT_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates tax rate percentage.
    This function receives year data and sales data list and provision for income tax.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - Provision_of_IT_data: List of corresponding provision for income tax.
    
    Returns:
    - A list of floats representing the tax rate percentage for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(Provision_of_IT_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, Provision_of_IT_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_IT_data, sorted_sales = zip(*data_sorted)

    yoy_Tax_Rate_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            tax_percentage = (sorted_IT_data[i] / sorted_sales[i]) * 100
            yoy_Tax_Rate_percentage.append(tax_percentage)
        else:
            yoy_Tax_Rate_percentage.append(None)  # If data is missing or invalid

    return yoy_Tax_Rate_percentage

In [172]:
tools = [Extract_Tool,Growth_Tool, GrossmmarginpercTool,ExpensespercTool, TaxrateTool]

In [173]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=key_string)

In [174]:
prompt = hub.pull("hwchase17/structured-chat-agent")

/Users/gunik/anaconda3/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [175]:
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)

In [176]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

In [177]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

In [178]:
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Extract_Tool and Growth_Tool"
memory.chat_memory.add_message(SystemMessage(content=initial_message))


In [192]:
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))



> Entering new AgentExecutor chain...
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Provision for Income Taxes",
    "c": "2020",
    "d": "2023"
  }
}([9680, 14527, 19300, 16741], [2020, 2021, 2022, 2023]){
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Net Sales",
    "c": "2020",
    "d": "2023"
  }
}([274515, 365817, 394328, 383285], [2020, 2021, 2022, 2023]){
  "action": "TaxrateTool",
  "action_input": {
    "Provision_of_IT_data": [9680, 14527, 19300, 16741],
    "year_data": [2020, 2021, 2022, 2023],
    "sales_data": [274515, 365817, 394328, 383285]
  }
}[3.5262189679981057, 3.971111238679449, 4.8944026292832365, 4.367768109892117]Could not parse LLM output: The tax rate percentages for Apple from 2020 to 2023 are as follows:
- 2020: 3.53%
- 2021: 3.97%
- 2022: 4.89%
- 2023: 4.37%
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILUREInvalid or incomplete respons

In [180]:
# !pip install pydantic==1.10.11



In [181]:
# prompt = hub.pull("hwchase17/openai-tools-agent")

In [182]:
# agent = create_tool_calling_agent(
#     llm=llm,  # Language model to use
#     tools=tools,  # List of tools available to the agent
#     prompt=prompt,  # Prompt template to guide the agent's responses
# )

In [183]:
# agent_executor = AgentExecutor.from_agent_and_tools(
#     agent=agent,  # The agent to execute
#     tools=tools,  # List of tools available to the agent
#     verbose=True,  # Enable verbose logging
#     handle_parsing_errors=True,  # Handle parsing errors gracefully
# )

In [184]:
# response = agent_executor.invoke({"input": " extract net sales of Apple 2019 to 2023 "})

In [185]:
# response = agent_executor.invoke({"input": "generate some insights of sales growth of apple"})

In [186]:
# print(response)